# Проект для интернет-магазина

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

Значение метрики качества *F1* должно быть не меньше 0.75. 

## Подготовка

In [1]:
!pip install catboost
!pip install langid

     |████████████████████████████████| 1.9 MB 1.2 MB/s eta 0:00:01
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941189 sha256=f1293413d07cd9ff13d82cedb8c99eab0ed5ee0826bbf4475831d3324540a0cf
  Stored in directory: /home/jovyan/.cache/pip/wheels/93/95/a9/c292c9dd8cadb8f2359f1670ff198a40d47167b0be3236e1c8
Successfully built langid


In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from sklearn.utils import shuffle
from catboost import CatBoostClassifier
import langid

In [3]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
df.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Можно отметить высокую степень дисбаланса классов.

Определим язык, на котором написаны комментарии. От этого будет зависить, какими инструментами будем лемматизировать текст

Напишем функцию, которая будет возвращать язык переданного ей текста, и применим ее к столбцу с комментариями

In [7]:
def language_detector(row):
  return langid.classify(row['text'])[0]

df['language'] = df.apply(language_detector, axis=1)

In [8]:
df.head()

,Unnamed: 0,text,toxic,language
0,0,Explanation\nWhy the edits made under my usern...,0,en
1,1,D'aww! He matches this background colour I'm s...,0,en
2,2,"Hey man, I'm really not trying to edit war. It...",0,en
3,3,"""\nMore\nI can't make any real suggestions on ...",0,en
4,4,"You, sir, are my hero. Any chance you remember...",0,en


In [9]:
df.language.value_counts()

en    155664
fr       343
de       341
es       288
it       270
       ...  
uk         1
mk         1
bg         1
or         1
ne         1
Name: language, Length: 85, dtype: int64

Как видно, язык написания абсолютного большинства комментариев определен как англйиский. 

Напишем функцию, которая очищаяет и лемматизирует текст.

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
def lemmatize_text(row):
  clear_text = ' '.join(re.sub(r'[^a-zA-Z ]', ' ', row['text']).split())
  tokens = nltk.word_tokenize(clear_text)
  tokens_lemmatized = []
  for token in tokens:
    tokens_lemmatized.append(lemmatizer.lemmatize(token))
  return str.lower(' '.join(tokens_lemmatized))  

In [12]:
df['lemmas'] = df.apply(lemmatize_text, axis=1)

In [13]:
df.head()

,Unnamed: 0,text,toxic,language,lemmas
0,0,Explanation\nWhy the edits made under my usern...,0,en,explanation why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,en,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,en,hey man i m really not trying to edit war it s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,en,more i can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,en,you sir are my hero any chance you remember wh...


Выделим фичи и таргет, затем поделим их на обучающую и тестовую выборки

In [14]:
features = df['lemmas']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.1, random_state=1)

Определим набор стоп-слов, которые будут исключены при векторизации текста

In [15]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Обучим векторайзер на обучающей выборке, затем преобразуем им обучающую и тестовую выборки

In [16]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

train_features_tf_idf = tf_idf.fit_transform(features_train)
test_features_tf_idf = tf_idf.transform(features_test)

## Обучение

### Логистическая регрессия

In [17]:
model = LogisticRegression()
model.fit(train_features_tf_idf, target_train)
predictions_logistic = model.predict(test_features_tf_idf)
print('F1-мера для модели логистической регрессии составила {:.2f}'.format(
    f1_score(target_test, predictions_logistic)))

F1-мера для модели логистической регрессии составила 0.73


Ранее мы обратили внимание, что классы в датасете несбалансированы. Попробуем обучить модель со взвешенными классами.

In [18]:
model = LogisticRegression(class_weight='balanced', max_iter=300)
model.fit(train_features_tf_idf, target_train)
predictions_logistic_balanced_weight = model.predict(test_features_tf_idf)
print('F1-мера для модели логистической регрессии со взвешенными классами составила {:.2f}'.format(
    f1_score(target_test, predictions_logistic_balanced_weight)))

F1-мера для модели логистической регрессии со взвешенными классами составила 0.75


### Catboost

In [19]:
features_cb = df[['lemmas']]
target_cb = df[['toxic']]

Поделим выборки на обучающую, валидационную и тестовую.

In [20]:
features_cb_train, features_cb_, target_cb_train, target_cb_ = train_test_split(features_cb, target_cb, test_size=.2, random_state=1)
features_cb_valid, features_cb_test, target_cb_valid, target_cb_test = train_test_split(features_cb_, target_cb_, test_size=.5, random_state=1)

In [21]:
%%time
model = CatBoostClassifier(iterations=500, learning_rate=0.03, depth=10)
model.fit(features_cb_train, target_cb_train, 
          eval_set=(features_cb_valid, target_cb_valid),
          text_features=['lemmas'], verbose=50)

0:	learn: 0.6508692	test: 0.6505408	best: 0.6505408 (0)	total: 4.6s	remaining: 38m 15s
50:	learn: 0.1546363	test: 0.1520313	best: 0.1520313 (50)	total: 3m 48s	remaining: 33m 32s
100:	learn: 0.1337030	test: 0.1326034	best: 0.1326034 (100)	total: 7m 30s	remaining: 29m 40s
150:	learn: 0.1264278	test: 0.1266953	best: 0.1266953 (150)	total: 11m 14s	remaining: 25m 58s
200:	learn: 0.1216829	test: 0.1232855	best: 0.1232855 (200)	total: 15m 1s	remaining: 22m 21s
250:	learn: 0.1180337	test: 0.1209860	best: 0.1209860 (250)	total: 18m 49s	remaining: 18m 40s
300:	learn: 0.1149801	test: 0.1193282	best: 0.1193282 (300)	total: 22m 33s	remaining: 14m 54s
350:	learn: 0.1123767	test: 0.1180177	best: 0.1180177 (350)	total: 26m 18s	remaining: 11m 10s
400:	learn: 0.1102120	test: 0.1170833	best: 0.1170833 (400)	total: 30m 4s	remaining: 7m 25s
450:	learn: 0.1087425	test: 0.1164631	best: 0.1164631 (450)	total: 33m 52s	remaining: 3m 40s
499:	learn: 0.1064797	test: 0.1156481	best: 0.1156481 (499)	total: 37m 32s	

In [22]:
predictions_cb = model.predict(features_cb_test)
print('F1-мера для модели CatBoost составила {:.2f}'.format(
    f1_score(target_cb_test, predictions_cb)))

F1-мера для модели CatBoost составила 0.77


## Выводы

**Выполнен анализ работы моделей логистической регрессии и CatBoost на классификации текстовых комментариев. Модель логистической регрессии показала более слабый результат, но нужно отметить высокую скорость ее работы. Модель CatBoost показала хороший результат, но потребовала значительные траты времени и ресурсов на обучение. Помимо этого, преимущество модели CatBoost заключается в том, что она работает непосредственно с текстовыми признаками, без необходимости в их векторизации.**